<a href="https://colab.research.google.com/github/nadia-maarfavi/Video.Analysis/blob/main/VideoShotsInfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import subprocess
import numpy as np
from PIL import Image
import pandas as pd
from numpy import asarray
import os
import glob
from scipy import stats
import cv2
import time

In [2]:
filename = "/content/drive/MyDrive/Video/Sample1.mp4"
#Path to the video

In [3]:
# This function will read the video and check the frame type.

def get_frame_types(video_fn):
    command = 'ffprobe -v error -show_entries frame=pict_type -of default=noprint_wrappers=1'.split()
    out = subprocess.check_output(command + [video_fn]).decode()
    frame_types = out.replace('pict_type=','').split()
    return zip(range(len(frame_types)), frame_types)

In [4]:
""" This function takes a video as input and initially displays key video parameters,
such as the frames per second, total number of frames, and duration.
Subsequently, for each keyframe (also known as an I-frame),
 it prints both the timestamp and the frame number.
Finally, the function saves each keyframe as an image file."""

def save_i_keyframes(video_fn):
    KeyframesNo = []
    Keyframestimes =[]
    video = cv2.VideoCapture(video_fn)
    fps = video.get(cv2.CAP_PROP_FPS)
    print("Frames Per Second:....", fps)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print("Number of total frames:.....", frame_count)
    duration = frame_count / fps
    print("Video Duration....", duration)

    frame_types = get_frame_types(video_fn)

    i_frames = [x[0] for x in frame_types if x[1]=='I']
    if i_frames:
        basename = os.path.splitext(os.path.basename(video_fn))[0]
        cap = cv2.VideoCapture(video_fn)
        print("Number of i frames:....", len(i_frames))

        for frame_no in i_frames:
            KeyframesNo.append(frame_no)
            timestamp = frame_no / fps
            Keyframestimes.append(timestamp)
        cap.release()
    else:
        print ('No I-frames in '+video_fn)


    return KeyframesNo, Keyframestimes, duration

In [5]:
''' This function accepts a video input and returns the number of shots detected,
along with the average and variance of the time between shots.'''

def ShotInfo(videopath):
    Difference = []
    Features = []
    frameno, frametime, durations = save_i_keyframes(videopath)
    ShotNo = len(frametime)

    frametime = np.asarray(frametime)

    i= 0
    while (i < ShotNo-1):
        diff = frametime[i+1] - frametime[i]
        Difference.append(diff)
        i+=1


    MeanShot = np.asarray(Difference).mean()
    VarShot = np.var(np.asarray(Difference))

    Features.append([ShotNo,MeanShot,VarShot])

    return Features

In [6]:
videopath = "/content/drive/MyDrive/Video/Sample1.mp4"

ShotFeatures = ShotInfo(videopath)
print(ShotFeatures)

Frames Per Second:.... 23.976023976023978
Number of total frames:..... 216
Video Duration.... 9.008999999999999
Number of i frames:.... 4
[[4, 2.419083333333333, 0.02435419097222229]]
